# SECTION 09: SQL DATABSES

## SQL Island
- You wash up on the shores of a mysterious island and must use your wits and SQL skills to survive.

https://sql-island.informatik.uni-kl.de/

# SQL DATABASES

## LEARNING OBJECTIVES:

- Understand what a database is and how it is different than a DataFrame/Excel sheet.
- Understand how to read database map
    - Primary keys vs forgein keys
- Understand how to select, filter, order, and group data using SQL
- Understand the different types of Joins 


SECTION 09: SQL DATABASES


SQL is designed to work with **relational data**. This really just means pieces of data that are **related to eachother**. In the example above, data on the `employees` table has some relationship to data on the `offices` table.

Each table has a **primary key** (like a DataFrame index), with a unique index for each row in the database. The name of the primary key is preceded by an asterix (\*). 

Columns that are the **primary key one on table** can also appear on **other tables**. Then it is refered to as a **foreign key** aka the primary key from a different ("foreign") table. 

<img src="https://raw.githubusercontent.com/learn-co-students/dsc-sql-introduction-online-ds-sp-000/master/images/Database-Schema.png">

## ⨠ Q: Why do we need databases? Why can't we just use a bunch of Pandas DataFrames?
- connectivity with large amount of users
- much more efficient
- No duplicate/redundant data, everything stored as unique values in a table.


## Querying Databases - `SELECT`ing data




- NOTE: SQL queries dot not _have_ to be all-caps, but it is a convention to help differentiate sql syntax versus names of tables/columns.

To retrieve data from one or more tables you usually use a `SELECT` statement. 

```
SELECT col1, col2, col3
FROM table
WHERE records match criteria
LIMIT 100;
```

Another similar query to preview all of the columns from the first 5 rows of a table would look like this:  

```SQL
SELECT *
FROM table
LIMIT 5;
```

Notice how all of these statements:
1.  **start with the `SELECT`**
2. followed by **what you want to select**. Separate multiple column names separated by a `,` 
3. Then specify where the data is coming `FROM` followed by the table name. 
4. **Afterward, you can provide conditions such as filters or sorting**. To demonstrate, here's a more complex example where you could preview the 10 most expensive payments received.

```SQL
SELECT *
FROM payments
ORDER BY amount DESC
LIMIT 10;
```

In [17]:
# from google.colab import drive
# drive.mount('/content/drive')

## SQL with `sqlite3`

<img src="https://raw.githubusercontent.com/learn-co-students/dsc-sql-introduction-online-ds-sp-000/master/images/Database-Schema.png" width=500>

## FILTERING AND ORDERING

- `ORDER BY` - `DESC`/`ASC`
- `LIMIT`
- `BETWEEN`
- `NULL`
- `COUNT`
- `GROUP BY`

## GROUPING DATA WITH SQL

In [18]:
db = 'datasets/data.sqlite'
import sqlite3
import pandas as pd 
conn = sqlite3.connect(db)
cur = conn.cursor()

- Like we do with Pandas, we can use GROUP BY statements in SQL and then apply **aggregate functions:**
    - `COUNT`
    - `MAX`
    - `MIN`
    - `SUM`
    - `AVG`

In [19]:
cur.execute("""SELECT city, COUNT(employeeNumber)
FROM offices 
JOIN employees
USING(officeCode)
GROUP BY city
ORDER BY count(employeeNumber) DESC;""")

df = pd.DataFrame(cur.fetchall())

df.columns = [x[0] for x in cur.description]

df.head()

,city,COUNT(employeeNumber)
0,San Francisco,6
1,Paris,5
2,Sydney,4
3,Boston,2
4,London,2


## ALIASING

- can assign a temporary name to data being imported
- Useful for `JOIN`,`GROUP BY`, and aggregates.

In [20]:
cur.execute("""SELECT city, COUNT(employeeNumber) AS numEmployees
               FROM offices
               JOIN employees
               USING(officeCode)
               GROUP BY 1
               ORDER BY numEmployees DESC;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,city,numEmployees
0,San Francisco,6
1,Paris,5
2,Sydney,4
3,Boston,2
4,London,2


In [21]:
cur.execute("""SELECT customerName,
               COUNT(customerName) AS number_purchases,
               MIN(amount) AS min_purchase,
               MAX(amount) AS max_purchase,
               AVG(amount) AS avg_purchase,
               SUM(amount) AS total_spent
               FROM customers
               JOIN payments
               USING(customerNumber)
               GROUP BY 1
               ORDER BY SUM(amount) DESC;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

98


,customerName,number_purchases,min_purchase,max_purchase,avg_purchase,total_spent
0,Euro+ Shopping Channel,13,116208.40,65071.26,55056.844615,715738.98
1,Mini Gifts Distributors Ltd.,9,101244.59,85410.87,64909.804444,584188.24
2,"Australian Collectors, Co.",4,44894.74,82261.22,45146.267500,180585.07
3,Muscle Machine Inc,4,20314.44,58841.35,44478.487500,177913.95
4,"Dragon Souveniers, Ltd.",4,105743.00,44380.15,39062.757500,156251.03


## The `WHERE` Clause



In general, the `WHERE` clause filters query results by some condition. As you are starting to see, you can also combine multiple conditions.

- 
```python
cur.execute("""SELECT * FROM customers WHERE city = 'Boston' OR city = 'Madrid';""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df
```


Two additional keywords that you can use to refine your searches are the `ORDER BY` and `LIMIT` clauses. The order by clause allows you to sort the results by a particular feature. For example, you could sort by the `customerName` column if you wished to get results in alphabetical order. By default, `ORDER BY` is ascending. So, as with the above example, if you want the opposite, use the additional parameter `DESC`. Finally, the limit clause is typically the last argument in a SQL query and simply limits the output to a set number of results.



## The `HAVING` clause

 The `HAVING` clause works similarly to the `WHERE` clause, except it is used to filter data selections on conditions **after** the `GROUP BY` clause.

In [22]:
cur.execute("""SELECT city, COUNT(customerNumber) AS number_customers
               FROM customers
               GROUP BY 1
               HAVING COUNT(customerNumber)>=5;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

2


,city,number_customers
0,Madrid,5
1,NYC,5


## Combining `WHERE` and `HAVING`

We can also use the `WHERE` and `HAVING` clauses in conjunction with each other for more complex rules.

- For example, let's say we want a list of customers who have made at least 3 purchases of over 50K each.

In [23]:
cur.execute("""SELECT customerName,
               COUNT(amount) AS number_purchases_over_50K
               FROM customers
               JOIN payments
               USING(customerNumber)
               WHERE amount >= 50000
               GROUP BY 1
               HAVING count(amount) >= 3
               ORDER BY count(amount) DESC;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

53


,customerName,number_purchases_over_50K
0,Euro+ Shopping Channel,13
1,Mini Gifts Distributors Ltd.,9
2,"Anna's Decorations, Ltd",4
3,"Australian Collectors, Co.",4
4,Baane Mini Imports,4


In [24]:
cur.execute("""SELECT customerName,
               COUNT(amount) AS number_purchases_over_50K
               FROM customers
               JOIN payments
               USING(customerNumber)
               WHERE amount >= 50000
               GROUP BY 1
               HAVING number_purchases_over_50K >= 3
               ORDER BY count(amount) DESC;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

53


,customerName,number_purchases_over_50K
0,Euro+ Shopping Channel,13
1,Mini Gifts Distributors Ltd.,9
2,"Anna's Decorations, Ltd",4
3,"Australian Collectors, Co.",4
4,Baane Mini Imports,4


## `JOIN` STATEMENTS

<img src="https://raw.githubusercontent.com/learn-co-students/dsc-sql-introduction-online-ds-sp-000/master/images/Database-Schema.png">

### Task: Displaying product details along with order details

Let's say you need to generate some report that includes details about products from orders. To do that, we would need to take data from multiple tables in a single statement. 

In [9]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('datasets/data.sqlite')
cur = conn.cursor()

In [10]:
cur.execute("""SELECT * 
               FROM orderdetails
               JOIN products
               ON orderdetails.productCode = products.productCode
               LIMIT 10;
               """)
df = pd.DataFrame(cur.fetchall()) #Take results and create dataframe
df.columns = [i[0] for i in cur.description]
df.head()

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber,productCode,productName,productLine,productScale,productVendor,productDescription,quantityInStock,buyPrice,MSRP
0,10100,S18_1749,30,136.00,3,S18_1749,1917 Grand Touring Sedan,Vintage Cars,1:18,Welly Diecast Productions,This 1:18 scale replica of the 1917 Grand Tour...,2724,86.70,170.00
1,10100,S18_2248,50,55.09,2,S18_2248,1911 Ford Town Car,Vintage Cars,1:18,Motor City Art Classics,"Features opening hood, opening doors, opening ...",540,33.30,60.54
2,10100,S18_4409,22,75.46,4,S18_4409,1932 Alfa Romeo 8C2300 Spider Sport,Vintage Cars,1:18,Exoto Designs,This 1:18 scale precision die cast replica fea...,6553,43.26,92.03
3,10100,S24_3969,49,35.29,1,S24_3969,1936 Mercedes Benz 500k Roadster,Vintage Cars,1:24,Red Start Diecast,This model features grille-mounted chrome horn...,2081,21.75,41.03
4,10101,S18_2325,25,108.06,4,S18_2325,1932 Model A Ford J-Coupe,Vintage Cars,1:18,Autoart Studio Design,This model features grille-mounted chrome horn...,9354,58.48,127.13


## The `USING` clause



A more concise way to join the tables, if the column name is identical, is the `USING` clause. Rather then saying on `tableA.column = tableB.column` we can simply say `using(column)`. Again, this only works if the column is **identically named** for both tables.

## TYPES OF JOINS



- Joins may be:
    - INNER (default)
    - OUTER
    - LEFT 
    - RIGHT
<img src="https://raw.githubusercontent.com/learn-co-students/dsc-join-statements-online-ds-sp-000/master/images/venn.png">


## Primary vs Foreign Keys
- primary key: unique identifier / index for a table.
- forgein key: A primary key from another table that can match data in the current table.


### One-to-One, One-to-many, many-to-many Joins


## SQL Subqueries

```python
cur.execute("""SELECT lastName, firstName, officeCode
               FROM employees
               WHERE officeCode IN (SELECT officeCode
                                    FROM offices 
                                    WHERE country = "USA");
                                    """)
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df
```

## SQL WITH PANDAS

### Using `.query()`

Pandas DataFrames come with a built-in query method, which allows you to get information from DataFrames quickly without using the cumbersome slicing syntax.  

See the following examples:

```python
# Getting Data using slicing syntax
foo_df = bar_df[bar_df[bar_df['Col_1'] > bar_df['Col_2']]]

# Using The query method
foo_df = bar_df.query("Col_1 > Col_2")

# These two lines are equivalent!
```

Note that if you want to use `and` and `or` statements with the `.query()` method, you'll need to use `"&"` and `"|"` instead.

```python
foo_df = bar_df.query("Col_1 > Col_2 & Col_2 <= Col_3")
```

## Using SQL with Pandas


- Pandas DataFrames have a method called `.query()`
- This allows us to use SQL-like commands to reference data.
```python
## Normal Pandas Syntax
foo_df = bar_df.loc[bar_df['Col_1']>bar_df['Col_2']]
```

```python
## Using .query()
foo_df = bar_df.query("Col_1 > Col_2")
```
- How to use:
    - Enter the querty as a single string, using just column names to reference data.
    - To use and/or statements, use `&` and `|`, respectively

```python
foo_df = bar_df.query("Col_1 > Col_2 & Col_2 <= Col_3")
```

## Using SQL syntax with `pandasql`



Since SQL is such a powerful, comfortable tool for Data Scientists, some people had the bright idea of creating a library that lets users query DataFrames using SQL-style syntax.  This library is called [pandasql](https://pypi.org/project/pandasql/).

We can install `pandasql` using the bash command `pip install pandasql`.

### Importing pandasql

In order to use `pandasql`, we need to start by importing a `sqldf` object from `pandasql`

```python
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
```

### Writing Queries
```python
q = """SELECT
        m.date, m.beef, b.births
     FROM
        meats m
     INNER JOIN
        births b
           ON m.date = b.date;"""

results = pysqldf(q)

```

## SQL Data Types


- Data types in SQLite3:
    - https://www.sqlite.org/datatype3.html
    
- Data types:
    - TEXT
    - INTEGER
    - REAL
    - BLOB
    - NULL


Since SQL is such a powerful, comfortable tool for Data Scientists, some people had the bright idea of creating a library that lets users query DataFrames using SQL-style syntax.  This library is called [pandasql](https://pypi.org/project/pandasql/).

We can install `pandasql` using the bash command `pip install pandasql`.

#### Importing pandasql

In order to use `pandasql`, we need to start by importing a `sqldf` object from `pandasql`

```python
from pandasql import sqldf
```


Next, it's helpful to write a lambda function that will make it quicker and easier to write queries.  Normally, you would have to pass in the global variables every time we use an object.  In order to avoid doing this every time, here's how to write a lambda that does this for you:

```python
pysqldf = lambda q: sqldf(q, globals())
```

#### Writing Queries

To write a query, you just format it as a multi-line string!

```python
q = """SELECT
        m.date, m.beef, b.births
     FROM
        meats m
     INNER JOIN
        births b
           ON m.date = b.date;"""
```

In order to query DataFrames, you can just pass in the query string you've created to our `sqldf` object that you stored in `pysqldf`.  This will return a DataFrame.  

```python
results = pysqldf(q)
```



## SQL Data Types
- Data types in SQLite3:
    - https://www.sqlite.org/datatype3.html
    
- Data types:
    - TEXT
    - INTEGER
    - REAL
    - BLOB
    - NULL
    


## DATABASE ADMIN 101

In [11]:
db = 'datasets/pets_database.db'
import sqlite3
conn = sqlite3.connect(db)
cur = conn.cursor()

In [12]:
cur.execute("""CREATE TABLE cats (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    breed TEXT)
    """)

OperationalError: table cats already exists

In [14]:
# insert Maru into the pet_database.db here
cur.execute('''INSERT INTO cats (name, age, breed) 
                  VALUES ('Maru', 3, 'Scottish Fold');
            ''')

In [15]:
cur.fetchall()

[]